## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [5]:
%load_ext lab_black

In [6]:
import os
import pathlib

In [7]:
this_dir = pathlib.Path(os.path.abspath(""))

In [8]:
data_dir = this_dir / "data"

In [9]:
import pytz
import glob
import json
import requests
import pandas as pd
from datetime import datetime, date

## Download

In [25]:
POWER_BI_QUERY_URL = (
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata"
)
REQUEST_PARAMS = (("synchronous", "true"),)
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"d","Entity":"demo_region_count","Type":0},{"Name":"r","Entity":"region_lastmonth","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"mregion"},"Name":"demo_region_count.mregion"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"case_count"}},"Function":0},"Name":"Sum(demo_region_count.case_count)"},{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"zip2"},"Name":"demo_region_count.zip2"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"case_count"}},"Function":0},"Name":"Sum(region_lastmonth.case_count)"}],"Where":[{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"zip2"}}],"Values":[[{"Literal":{"Value":"null"}}]]}}}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"mregion"}}],"Values":[[{"Literal":{"Value":"\'Unknown\'"}}]]}}}}}],"OrderBy":[{"Direction":2,"Expression":{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"case_count"}},"Function":0}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3]}]},"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"Version":1},"ExecutionMetricsKind":1}}]},"QueryId":"","ApplicationContext":{"DatasetId":"2e66a584-6b5f-4c9f-bffd-12d35d2b8442","Sources":[{"ReportId":"40085a43-f04a-4a8b-ae72-c38ca32531a5","VisualId":"136278efd4c6f694b9b7"}]}}],"cancelQueries":[],"modelId":358821}'

In [26]:
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,da;q=0.8",
    "activityid": "07a2f3cb-ea39-4519-8ee0-b5605aa78145",
    "content-type": "application/json;charset=UTF-8",
    "requestid": "a5565ffd-b522-00a7-afe8-5c4f3a3ce843",
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-powerbi-resourcekey": "69a5588e-f8a1-4aef-aac5-ca27ee543007",
    "Referer": "https://app.powerbigov.us/",
    "Referrer-Policy": "strict-origin-when-cross-origin",
}
json_data = json.loads(REQUEST_BODY)

In [27]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/Users/sgreene/.local/share/virtualenvs/california-coronavirus-scrapers-o-Rl0Txv/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [28]:
data = response.json()

## Parse

In [29]:
timestamp = response.headers["Date"]

In [90]:
timestamp

'Thu, 05 May 2022 03:11:11 GMT'

In [31]:
ds = data["results"][0]["result"]["data"]["dsr"]["DS"][0]

In [40]:
zip_codes = ds["ValueDicts"]["D1"]

In [33]:
rows = ds["PH"][0]["DM0"]

In [64]:
confirmed_cases = []

cases = 0

for row in rows:
    # in some cases, data are encoded in separate keys
    keys = row.keys()
    # pull out the list containing values for most rows
    cells = row["C"]
    # second value in the list is expected to be an iterator/index
    index = cells[1]
    # if this key is present, the value for this place is identical to the previous row
    if "R" in keys:
        cases = cells[1]
        index = cells[0]
    # every other row has its value in the third position of the list
    else:
        cases = cells[2]
    # append values to master list
    confirmed_cases.append(
        (
            index,
            cases,
        )
    )

In [65]:
matched = [(zip_codes[index], cases) for index, cases in confirmed_cases]

In [66]:
df = pd.DataFrame(matched, columns=["zip_code", "confirmed_cases"])

Match up place names with zip codes

In [76]:
df = df.loc[(df.zip_code != "Other")]

In [77]:
zip_code_names = {
    "93902": "Prunedale",
    "93907": "Salinas",
    "95004": "Aromas",
    "95012": "Castroville",
    "95039": "Moss Landing",
    "95076": "Watsonville",
    "93920": "Big Sur",
    "93921": "Carmel",
    "93923": "Carmel",
    "93924": "Carmel Valley",
    "93933": "Marina",
    "93940": "Monterey",
    "93942": "Monterey",
    "93950": "Pacific Grove",
    "93953": "Pebble Beach",
    "93955": "Seaside",
    "93901": "Salinas",
    "93905": "Salinas",
    "93906": "Salinas",
    "93908": "Salinas",
    "93912": "Boronda",
    "93915": "Salinas",
    "93962": "Spreckels",
    "93450": "San Ardo",
    "93925": "Chualar",
    "93926": "Gonzales",
    "93927": "Greenfield",
    "93930": "King City",
    "93954": "San Lucas",
    "93960": "Soledad",
    "93922": "Carmel",
    "93426": "Bradley",
    "93451": "San Miguel",
}

In [78]:
df["area_name"] = df.zip_code.map(zip_code_names)

In [79]:
df["area_name"] = df["zip_code"] + ": " + df["area_name"]

In [87]:
df = df[(df.zip_code != "OtherPBS")]

Set column with county date

In [91]:
df["county_date"] = pd.to_datetime(timestamp)

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_88663/3877055356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["county_date"] = pd.to_datetime(timestamp)


In [92]:
df["county_date"] = df["county_date"].dt.tz_convert("US/Pacific")

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_88663/2921071059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["county_date"] = df["county_date"].dt.tz_convert("US/Pacific")


In [93]:
df["county_date"] = df["county_date"].dt.strftime("%Y-%m-%d")

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_88663/236788264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["county_date"] = df["county_date"].dt.strftime("%Y-%m-%d")


In [94]:
df.insert(0, "county", "Monterey")

Clean up for export

In [95]:
export_df = df[
    ["county", "area_name", "confirmed_cases", "county_date", "zip_code"]
].rename(columns={"area_name": "area", "zip_code": "zip"})

## Vet

In [100]:
try:
    assert not len(export_df) < 25
except AssertionError:
    raise AssertionError("Monterey County's zip code scraper is missing rows")

In [101]:
try:
    assert not len(export_df) > 25
except AssertionError:
    raise AssertionError("Monterey County's zip code scraper has more rows than before")

## Export

Set date

In [102]:
tz = pytz.timezone("America/Los_Angeles")

In [103]:
today = datetime.now(tz).date()

In [104]:
slug = "monterey"

In [105]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [106]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [107]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [108]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [109]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)